In [9]:
class Vehicle(object):
    def __init__(self, name, colour):
        self.name = name
        self.colour = colour

    class Crash(Exception):
        pass

    def __add__(self, value):
        if isinstance(value, Vehicle):
            raise self.Crash(f"You ({self.name}) crashed into {value.name}")
        raise NotImplementedError

    def __str__(self):
        return f"{self.colour} {self.name}"


In [10]:
class Car(Vehicle):
    def __init__(self, name, colour, doors):
        """ cars have doors """
        self.doors = int(doors)
        super().__init__(name, colour)


In [11]:
car_1 = Car("Hyundai i30", "blue", 5)
car_2 = Car("Fiat 500", "grey", 3)

In [12]:
car_2 + car_1

Crash: You (Fiat 500) crashed into Hyundai i30

In [13]:
type(car_1)

__main__.Car

In [14]:
class Motorcycle(Vehicle):
    # bikes don't have doors
    pass

In [15]:
bike_1 = Motorcycle("Kawasaki Ninja", "Green")

In [16]:
car_1 + bike_1

Crash: You (Hyundai i30) crashed into Kawasaki Ninja

In [19]:

class Trailer(Vehicle):

    can_carry = (Motorcycle,)

    def __init__(self, name, colour):
        self.contains = None
        super().__init__(name, colour)

    def __add__(self, value):
        if type(value) in self.can_carry:
            self.contains = value
            print(f"loaded {value.name}")
            return True
        super().__add__(value)

    def __str__(self):
        if self.contains:
            return f"{self.colour} Trailer {self.name} containing {str(self.contains)}"
        return f"{self.colour} Trailer {self.name}"


In [17]:
bike_1 + 3

NotImplementedError: 

In [20]:
tailer_1 = Trailer('rusty', 'rust')

In [21]:
tailer_1 + car_1

Crash: You (rusty) crashed into Hyundai i30

In [22]:
trailer_1 = tailer_1

In [23]:
trailer_1 + bike_1

loaded Kawasaki Ninja


True

In [24]:
trailer_1.contains

In [25]:
id(bike_1)

4579515392

In [26]:
trailer_1.contains.name

'Kawasaki Ninja'

In [27]:
class Truck(Car):

    can_tow = (Trailer,)

    def __init__(self, name, colour, doors):
        """ cars have doors """
        self.towing = None
        super().__init__(name, colour, doors)

    def __add__(self, value):
        if type(value) in self.can_tow:
            if not self.towing:
                self.towing = value
                print(f"hooked up {value.name}")
            else:
                print(f"already towing {self.towing.name}")
            return None
        super().__add__(value)

    def __str__(self):
        if self.towing:
            return f"{self.colour} Truck {self.name} towing {str(self.towing)}"
        return f"{self.colour} Truck {self.name}"




In [28]:
truck_1 = Truck('Dodge Ram', 'black', 4)

In [29]:
truck_1

In [30]:
str(truck_1)

'black Truck Dodge Ram'

In [31]:
truck_1 + trailer_1

hooked up rusty


In [32]:
truck_1.towing

In [33]:
str(truck_1.towing)

'rust Trailer rusty containing Green Kawasaki Ninja'

In [34]:
str(truck_1.towing.contains)

'Green Kawasaki Ninja'

In [35]:
truck_1.towing.name

'rusty'

In [36]:
truck_1.towing.contains.name

'Kawasaki Ninja'

In [37]:
dir(truck_1)

['Crash',
 '__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'can_tow',
 'colour',
 'doors',
 'name',
 'towing']

In [38]:
type(truck_1).__bases__

(__main__.Car,)

In [39]:
type(truck_1).__mro__

(__main__.Truck, __main__.Car, __main__.Vehicle, object)

In [107]:
class Ship(Vehicle):
    capacity = 1
    def __init__(self, name, colour):
        self.contains = []
        self.afloat = True
        super().__init__(name,colour)

    class Sink(Exception):
        pass

    def __add__(self, value):
        if not self.afloat:
            raise ValueError("can not load item to sunken ship")
        if not isinstance(value, Vehicle):
            raise ValueError("Not a vehicle")
        if len(self.contains) < self.capacity:
            self.contains.append(value)
            print(f"loaded {value.name} onto {self.name}")
        else:
            self.afloat = False
            raise self.Sink(f"you sunk my {str(self)}")
            
    def __sub__(self, idx):
        """ remove a thing from the ship"""
        x = self.contains.pop(idx)
        print(f"removed {x.name} from {self.name}")
        
    def __float__(self):
        self.afloat = True
        print(f"floated {self.name}")
        return float(0)
        

    @property
    def inventory(self):
        for idx, thing in enumerate(self.contains):
            print(f"slot {idx:<2} {str(thing)}")
        

In [108]:
class Ferry(Ship):
    # naval architect says it can carry 10
    capacity = 10


In [109]:
boaty_mc_boatface = Ferry("Boaty McBoatface", "Rainbow")

In [110]:
str(boaty_mc_boatface)

'Rainbow Boaty McBoatface'

In [111]:
boaty_mc_boatface + truck_1

loaded Dodge Ram onto Boaty McBoatface


In [112]:
boaty_mc_boatface.contains

In [64]:
boaty_mc_boatface.inventory

slot 0  black Truck Dodge Ram towing rust Trailer rusty containing Green Kawasaki Ninja


In [47]:
boaty_mc_boatface + 5

ValueError: Not a vehicle

In [48]:
boaty_mc_boatface.contains[0].towing.contains.name

'Kawasaki Ninja'

In [49]:
type(boaty_mc_boatface).__mro__

(__main__.Ferry, __main__.Ship, __main__.Vehicle, object)

In [50]:
str(boaty_mc_boatface)

'Rainbow Boaty McBoatface'

In [51]:
boaty_mc_boatface.__str__

<bound method Vehicle.__str__ of <__main__.Ferry object at 0x113913ac0>>

In [52]:
boaty_mc_boatface.__str__()

'Rainbow Boaty McBoatface'

In [53]:
dir(boaty_mc_boatface)

['Crash',
 'Sink',
 '__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'afloat',
 'capacity',
 'colour',
 'contains',
 'inventory',
 'name']

In [113]:
for x in range(10):
    boaty_mc_boatface + Car('Generic Car', "vanilla", 4)

loaded Generic Car onto Boaty McBoatface
loaded Generic Car onto Boaty McBoatface
loaded Generic Car onto Boaty McBoatface
loaded Generic Car onto Boaty McBoatface
loaded Generic Car onto Boaty McBoatface
loaded Generic Car onto Boaty McBoatface
loaded Generic Car onto Boaty McBoatface
loaded Generic Car onto Boaty McBoatface
loaded Generic Car onto Boaty McBoatface


Sink: you sunk my Rainbow Boaty McBoatface

In [114]:
boaty_mc_boatface.inventory

slot 0  black Truck Dodge Ram towing rust Trailer rusty containing Green Kawasaki Ninja
slot 1  vanilla Generic Car
slot 2  vanilla Generic Car
slot 3  vanilla Generic Car
slot 4  vanilla Generic Car
slot 5  vanilla Generic Car
slot 6  vanilla Generic Car
slot 7  vanilla Generic Car
slot 8  vanilla Generic Car
slot 9  vanilla Generic Car


In [120]:
boaty_mc_boatface + car_1

Sink: you sunk my Rainbow Boaty McBoatface

In [115]:
boaty_mc_boatface.afloat

False

In [90]:
boaty_mc_boatface - 9

removed Generic Car from Boaty McBoatface


In [117]:
boaty_mc_boatface.afloat

True

In [77]:
boaty_mc_boatface == 1

False

In [122]:
float(boaty_mc_boatface)

floated Boaty McBoatface


0.0

In [123]:
str(boaty_mc_boatface)

'Rainbow Boaty McBoatface'

In [124]:
boaty_mc_boatface.contains